In [1]:
import spacy
import os
import xml.etree.ElementTree as ET
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from spacy import displacy
import scipy.stats

from spacy.tokens import Doc
# Tokenize only on whitespace to compare spacy tagger with sparv's tags
class WhitespaceTokenizer:
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        words = text.split(" ")
        return Doc(self.vocab, words=words)

nlp = spacy.load(os.path.join("..", "sv_model_upos", "sv_model0", "sv_model_upos0-0.0.0"))
#nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

In [ ]:
# Look at distribution of sentence lengths across languages/times
"""
    Process: pass in the function that will process individual sentences
    Returns whatever process returns
"""
def traverseSwed(filepath, process, relative=False):
    parser = ET.iterparse(file_path, events=('start','end'))
    parser = iter(parser)
    event, root = next(parser)

    words = ""
    pos = []
    sum_overlaps= 0 
    num_para = 0
    i = 0
    ag = []
    for event, elem in parser:
        if elem.tag == "w":
            if event == "end":
                #print(elem.attrib['pos'])
                words +=  elem.text + " "
                pos.append(elem.attrib['pos'])
                elem.clear()
            root.clear()
        if elem.tag == "sentence" and event=="end":
            words = words.strip()
            if len(words) == 0: continue
            doc = nlp(words)
            tok = list(doc.sents)[0].root
            if process != sentenceLength and relative:
                ag += [process(tok)/len(doc)]
            else: 
                ag += [process(tok)]
            
            words = ""
            pos = []
            i += 1
        if i == 10000: break
    return ag
            
def traverseFin(filepath, process, relative=False):
    ag = []
    num_sentences = 0
    with open(file_path) as reader:
        words = []
        pos = []
        for line in reader:
            if num_sentences == 10000: return sum_overlaps / num_sentences
            # Start of sentence
            if line[:9] == "<sentence": 
                #print("start sentence")
                words = []
                pos_spacy=pos_sparv=pos=[]
            # End of sentence
            if line[:11] == "</sentence>": 
                if(len(words)) == 0: continue
                sent = " ".join(words)
                # print(words)
                doc = nlp(sent)
                tok = list(doc.sents)[0].root
                if process != sentenceLength and relative: 
                    ag += [process(tok)/len(doc)]
                else: 
                    ag += [process(tok)]
                
                num_sentences += 1
            # Split up by sentence
            # If at a text line and not just tags
            if line[0] != "<": 
                wordInfo = line.split()
                #print(wordInfo)
                words += [wordInfo[0]]
                pos += [wordInfo[1]]
            if num_sentences == 10000: break
    
    return ag

def sentenceLength(tok):
    # print(doc, len(doc))
    return len(list(tok.children)) + 1

def treeDepth(tok):
    if len(list(tok.children)) == 0: return 0 
    
    depths = np.array([treeDepth(c) for c in tok.children]).flatten()
    return max(depths) + 1

def numLeafs(tok): 
    if len(list(tok.children)) == 0: return 1
    
    leafs = np.array([numLeafs(c) for c in tok.children]).flatten()
    return sum(leafs)

"""
    For a sentence with the root token tok, return the breakdown on the dependency relations
"""
def numComponents(tok): 
    